## Libraries

In [1]:
import requests
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import time
import concurrent.futures
from datetime import datetime, timedelta
import json

from bs4 import BeautifulSoup

-----
-----
-----

# Web scrapping test - IMDB

In [2]:
##### getting pages urls

##### ATTENTION: Process takes roughly 20 minutes !!

start_time = time.time()

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36"
}
# selecting all the genres in IMDB
genres = [
    'action', 'adventure', 'animation', 'biography',
    'comedy', 'crime', 'documentary', 'drama',
    'family', 'fantasy', 'film-noir', 'game-show',
    'history', 'horror', 'music', 'musical',
    'mystery', 'news', 'reality-tv', 'romance',
    'sci-fi', 'sport', 'talk-show', 'thriller',
    'war', 'western'
]

# Selecting movies with at least:
## Rating = 5.0
## Votes = 3000

url_start="https://www.imdb.com/search/title/?title_type=feature,tv_movie,short&user_rating=5.0,&num_votes=2000,&genres="
url_mid="&count=250&start="
url_end="&ref_=adv_nxt"

url_list = []

for genre in genres:
    ## looping through all the possible pages
    for i in range(1, 9952, 250):
        url = url_start + str(genre) + url_mid + str(i) + url_end
        
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        movie_items = soup.find_all("div", class_="lister-item-content")
        
        if not movie_items:
            break
        
        if url not in url_list:
            url_list.append(url)

    #    print(i)
  #  print(genre)
total_time = time.time() - start_time
print(f"Total execution time: {total_time} seconds")


KeyboardInterrupt



In [ ]:
###### getting movie urls

###### ATTENTION: Process takes roughly 20 minutes !!

start_time = time.time()

url_movie_list = []

i=1
for url in url_list:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    movie_items = soup.find_all("div", class_="lister-item-content")
    
    for movie_item in movie_items:
        movie_link = movie_item.find("a")
        movie_url = "https://www.imdb.com" + movie_link.get("href")
        
        if movie_url not in url_movie_list:
            url_movie_list.append(movie_url)
            
   # print(i)
    i=i+1
    
total_time = time.time() - start_time
print(f"Total execution time: {total_time} seconds")

In [ ]:
###### function to web scrape all the movie urls

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36"
}

def fetch_data(url_list):
    results = []
    batch_size = 100  # Number of URLs to process in each batch
    for i in range(0, len(url_list), batch_size):
        batch_urls = url_list[i:i+batch_size]
        
      #  j=1
        for url in batch_urls:

            response = requests.get(url, headers=headers)

            soup = BeautifulSoup(response.text, "html.parser")
            script_tag = soup.find("script", {"type": "application/ld+json"})
            
            if script_tag:
                json_data = json.loads(script_tag.string)

                # Extract required data
                try:
                    title = json_data.get("name")
                except:
                    title=np.nan
                try:
                    image = json_data.get("image")
                except:
                    image = np.nan
                try:
                    rating = json_data.get("aggregateRating", {}).get("ratingValue")
                except:
                    rating = np.nan
                try:
                    nr_rates = json_data.get("aggregateRating", {}).get("ratingCount")
                except:
                    nr_rates=np.nan
                try:
                    description = json_data.get("description")
                except:
                    description=np.nan
                try:
                    genres = json_data.get("genre")
                except:
                    genres = np.nan
                try:
                    pub_date = json_data.get("datePublished")
                except:
                    pub_date=np.nan
                try:
                    content_rating = json_data.get("contentRating")
                    director_data = json_data['director']
                    directors = ', '.join(director['name'] for director in director_data)
                except:
                    directors=np.nan
                try:
                    content_rating = json_data.get("contentRating")
                    creator_data = json_data['creator']
                    writers = ', '.join(creator['name'] for creator in creator_data if 'name' in creator)
                except:
                    writers=np.nan
            
            #new script
            script_tag = soup.find("script", {"type": "application/json"})
            
            if script_tag:
                json_data = json.loads(script_tag.string)

                try:
                    popularity = np.nan
                    if "props" in json_data and "pageProps" in json_data["props"] and "aboveTheFoldData" in json_data["props"]["pageProps"] and "meterRanking" in json_data["props"]["pageProps"]["aboveTheFoldData"] and "currentRank" in json_data["props"]["pageProps"]["aboveTheFoldData"]["meterRanking"]:
                        popularity = json_data["props"]["pageProps"]["aboveTheFoldData"]["meterRanking"]["currentRank"]
                except:
                    popularity = np.nan

                try:
                    actors = []
                    if "props" in json_data and "pageProps" in json_data["props"] and "mainColumnData" in json_data["props"]["pageProps"] and "cast" in json_data["props"]["pageProps"]["mainColumnData"] and "edges" in json_data["props"]["pageProps"]["mainColumnData"]["cast"]:
                        cast_edges = json_data["props"]["pageProps"]["mainColumnData"]["cast"]["edges"]
                        for edge in cast_edges:
                            actor_name = edge["node"]["name"]["nameText"]["text"]
                            actors.append(actor_name)  
                        actors_str = ', '.join(actors)
                    else:
                        actors_str=np.nan
                except:
                    actors_str=np.nan

            # Append the extracted data to the results list
            results.append({
                'url': url,
                'title': title,
                'image': image,
                'rating': rating,
                'nr_rates': nr_rates,
                'description': description,
                'genres': genres,
                'pub_date': pub_date,
                'content_rating': content_rating,
                'directors': directors,
                'writers': writers,
                'popularity': popularity,
                'actors': actors_str
            })

           # print(j)
           # j=j+1
                
    return results

In [ ]:
##### applying the function
##### ATTENTION: Process takes roughly 12 hours !!

start_time = time.time()

data = fetch_data(url_movie_list)

df = pd.DataFrame(data)

total_time = time.time() - start_time
print(f"Total execution time: {total_time} seconds")

df

In [ ]:
pd.set_option('display.max_colwidth', 100)

df["genres"] = df["genres"].astype(str)

df["genres"] = df["genres"].str.replace("[", "")
df["genres"] = df["genres"].str.replace("]", "")
df["genres"] = df["genres"].str.replace("'", "")

In [ ]:
df

In [ ]:
df.to_csv("imdb_movie_fetch.csv")

# ---- THE END ----

---
---
---
---

##### Actors birthdays

In [8]:
# Split the 'actors' column based on commas and create a list of names
names_list_actors = df['actors'].str.split(',')

# Extract names from each split and strip whitespace for actors
actors = [name.strip() for sublist in names_list_actors if isinstance(sublist, list) for name in sublist]

# Combine all unique names from directors, writers, and actors
unique_names = set(actors)
unique_names

{'Alaska L. McFadden',
 'Meredith MacNeill',
 'Ruth Trouncer',
 'Christopher Aydon',
 'Rick Marshall',
 'Cliff Woolner',
 'Branislav Fistric',
 'Shaun Prendergast',
 'Ignazio Roberto Daidone',
 'Melvin Van Peebles',
 'MacIntyre Dixon',
 'Christine Oesterlein',
 'Michael Starke',
 'Elie Wajeman',
 'Sharon M. Lewis',
 'Hyuk-ho Kwon',
 'Maísa Lacerda',
 'Eduardo MacGregor',
 'Minoru Hirano',
 'Jack Sergent-Shadbolt',
 'Marshall Lewy',
 'Michael Pizzuto',
 'Amad Jackson',
 'Víctor Alfonso Zárate Mendoza',
 'Robert Mika',
 'Aaron Schroeder',
 'William Tost',
 'Ralf Wolter',
 'Pavel Dobrovsky',
 'Sandra Ann Roberts',
 'Robert Ginnaven',
 'Navya La Shay',
 'Jonah Lees',
 'Jeremy Donaldson',
 'Ray Burnet',
 'Beate Finckh',
 'Jeremy Strong',
 'Nancy McLoughlin',
 'Lauren Sick',
 'Romain Lancry',
 'Fay Bainter',
 'Moritz Grove',
 'Louise Brooks',
 'Meenu Mishra',
 'Shaye Cogan',
 'Jennifer Coolidge',
 'Ashok Vyas',
 'Rocío Muñoz',
 'Sarah Brightman',
 'Don Coughlin',
 'Paul Sullivan Jr.',
 'Gijs

In [9]:
def get_birthdate_from_wikipedia(person_name):
    try:
        url = f"https://en.wikipedia.org/wiki/{person_name.replace(' ', '_')}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        infobox = soup.find('table', class_='infobox')
        birthdate_row = infobox.find('th', text='Born').parent
        birthdate = birthdate_row.find('span', class_='bday').text
        
      #  print(person_name)
        return birthdate
    except:
        return np.nan
     #   print(person_name)

In [10]:
### this process takes a LONG TIME !

df_birthdays=pd.DataFrame(unique_names, columns=['Name'])
df_birthdays['Birthdate'] = df_birthdays['Name'].apply(get_birthdate_from_wikipedia)

df_birthdays

Meredith MacNeill
Melvin Van Peebles
Michael Starke
Ralf Wolter
Beate Finckh
Fay Bainter
Louise Brooks
Shaye Cogan
Jennifer Coolidge
Sarah Brightman
Gijs Scholten van Aschat
Suthi Veerabhadra Rao
Ana Maria Popa
Carly Rae Jepsen
T-Pain
Robert Priest
Bernard Giraudeau
Carolyn Hennesy
Bruno Wolkowitch
Pauly Shore
Yoshio Harada
John Paul Ruttan
John Brumpton
Stephen Wheeler
Claudio Bigagli
Bruce Jarchow
Heinz Engelmann
Ronald Leigh-Hunt
Károly Makk
Alex Brightman
Acun Ilicali
Jordan Brower
Tia Carrere
Margot Rose
Paz Vega
Vinay Rai
Jules Asner
Harriet Toompere
Nancy Barrett
Alex Boye
Jennifer Lyons
Nana Bryant
Avery Brooks
Renée Jones
Caleb Landry Jones
Cassandra Peterson
Marlo Thomas
Paulo Tiefenthaler
Chynna Phillips
Patrick Fugit
Håkan Serner
Richard Karron
Sam Corlett
Sif Ruud
Jonathan Keltz
John Baskcomb
Renée Fokker
Mark Bellamy
Ram Charan
Jay Abdo
Cathy Jones
Swaminathan
Yayo Guridi
Jessica Parker Kennedy
Joan Gaspart
Scott Wood
Lisa Barbuscia
Julia Stiles
Steve Boone
Olivia Burnett

Valérie Donzelli
Frank McGlynn Sr.
Kathleen O'Malley
Lewis Wilson
Amy Yoder
Marjorie Rambeau
Paul Faivre
Belén Esteban
Raquel Torres
Enrico Nigiotti
Simon Greenall
Jack Wild
Nina Schultz
Joe Gatto
John Arcilla
Avraham Aviv Alush
Pedro Pascal
Gertan Klauber
Akira Kubo
Ramana Reddy
Im Ye-jin
Siegfried Fischbacher
Ravi Bhushan
Tungia Baker
Hina Dilpazir
Anthony Jeselnik
JC Chasez
Madison Smartt Bell
Lise Cutter
Tim Shadbolt
Yuki Tanaka
Jim Howick
Avis Bunnage
Elizabeth Henstridge
Charles Clary
Michelle Yeoh
Christopher Gorham
Hiroyuki Nagato
Fiona Glascott
Thurl Bailey
William Woodson
Fred Durst
Gisèle Halimi
Simone Valère
Niklas Hogner
Patrick Préjean
Roderic Noble
Edward Underdown
Guy Deghy
Miranda July
Jamie Denbo
Tihana Lazovic
Norman Snow
Alan Reed
Julian Kostov
Antonio Gasset
Toni Bertorelli
Dax Shepard
Forrest Lewis
Ben Mendelsohn
Joseph Castanon
Lola Gaos
Selema Masekela
Peter Haskell
Juan Diego Botto
Kiara Advani
Catherine Robbe-Grillet
Greg Cromer
Sen Wang
James Bolam
Mark Bento

Maria Doyle Kennedy
Brian Bowman
Tomoko Naraoka
Andy Karl
Steve Martin
Idil Firat
Dolores Heredia
Jason Bonham
Frankie Chan
Guillaume Cyr
Jack O'Leary
Victoria Carroll
Jacqueline Susann
Aditi Gupta
Monica Dolan
Sanjeev Kumar
Burt Douglas
François-Marie Banier
Frank Collison
Billy Joe Shaver
Josh Graham
Jermaine Crawford
Georges Corraface
Nae Caranfil
Lucie Mannheim
Robbie Magasiva
Jonathan Andersson
Adèle Castillon
Barbara McNair
Milton Reid
Edgar Kennedy
Om Bhutkar
Ben Gage
Sid Lucero
Zak Penn
Tamara Hope
Claudia Bryar
Julieta Cardinali
Arnold Korff
Dean Harens
Charlie David
Colleen Clinkenbeard
Marty Pollio
Brianna Brown
Zoltán Fábri
Ian Brown
Kim Sung-ryung
Marco Ferreri
Jason O'Mara
Annaleigh Ashford
Tom Chadbon
Grigoriy Dobrygin
Harley Venton
Richard Folmer
Emraan Hashmi
Glenn Close
Jordan Bridges
Empress Schuck
Kan Tokumaru
Ricou Browning
Giusi Merli
Eddie Garcia
Anne-Marie Duff
Isaac Hayes III
Carrie Coon
Count Prince Miller
Andrea Lowe
Betty Lou Keim
Tenika Davis
Charlie Thurst

Peter Tork
Barry Humphries
Jaime Ray Newman
Florencia Lozano
Dan Duryea
Leon Morgan
Nico Tortorella
Jerry Douglas
Grant Goodeve
Pete Holmes
Deborah Goodrich
Bokuzen Hidari
Desmond Young
Monique Rolland
Emily Holmes
Tao Tsuchiya
Aldo Nadi
Joanna Vanderham
Carlos PenaVega
Laurence Naismith
Andy Burrows
Felton Perry
Houseley Stevenson
Richard Levi
Prem Chopra
Lev Prygunov
Evelyne Buyle
Frank Fontaine
John Orcsik
Nellie Connally
Tseng Chang
Liam Redmond
Raghuvaran
Harry James
Kenneth Lonergan
Lauren Ambrose
Tom Wheeler
Heidy Forster
Lionel Murton
Roberta Valderrama
Kenneth Cope
Ian Lavender
Kaikala Satyanarayana
Zoe Chao
Ricky Gervais
John Longden
Indraneil Sengupta
Angela Moore
George M. Carleton
Bala Saravanan
Günter Strack
Rosebud Baker
Michael Hogben
Darby Camp
Philippe de Broca
Scott Michael Foster
Guillem Vives
Mushond Lee
Michael Elwyn
Patricia Allison
Kimora Lee Simmons
Elisabetta Rocchetti
Masato Harada
Justin Hall
Monica Barbaro
David Scheller
Jorge D'Elía
Assa Sylla
Emilio Buale

Harold Nicholas
Nick Romeo Reimann
Jared Keeso
Carlinhos Brown
Richard Dimitri
Axel Bauer
Tempe Pigott
Alberta Watson
George Siegmann
Bruce Springsteen
Red Skelton
Freida Pinto
Robert Byrd
Frank Shannon
Joe Rea
Amy Landecker
Mako Ishino
Will.i.am
Atsushi Arai
Navi Rawat
Jacques Perrin
Larry Sullivan
Julia Cho
Madeleine Arthur
Derrick De Marney
Katsunori Takahashi
Gustavo Sorola
Leo Ford
Jason Presson
Jordan Woods-Robinson
Shin Min-a
Debra Jo Rupp
Rica Fukami
Jesse Welles
Stéphane Julien
Crystal Allen
Konstantin Bogomolov
Crystal Rae
Paulo Avelino
Don Hudson
Olivia Wilde
Josh Hutcherson
John Sloman
Alma Jodorowsky
Dan Futterman
Minoru Terada
Salah Zulfikar
Sidney Toler
Cush Jumbo
Ami Aaröe
George Hamlin
George Hirsch
Shreechand Makhija
Brian Dietzen
Mao Ichimichi
Nacho Fresneda
Bridget Everett
Christopher Maleki
Daniyal Raheel
Cleo Moore
Rasmus Bjerg
Vladimir Kenigson
Shannon Tarbet
Ursula Andress
Terry Pheto
Andie MacDowell
Eric Lively
Ines Fancovic
Vincenzo Amato
Ottavia Piccolo
Els D

...

Jacob Brent
Aurora Miranda
Omar Sharif Jr.
Carmen Salinas
Trevor St. John
Andrew Divoff
G-Eazy
Ayumu Murase
Ken King
Gabrielle Miller
Ayesha Kapur
Christian Brando
Leo Borg
Brittany Furlan
Claudia Galli
Stuart Rogers
Jamie Johnston
Carl-Einar Häckner
Marjorie Gateson
Bhanu Uday
Anna Camp
Nando Cicero
Vivian Gibson
Rosie Perez
Abbey Lincoln
Kevin Loring
Brinke Stevens
Michel Saint-Denis
Julia Louis-Dreyfus
Shelley Duvall
John Travolta
Arturs Skrastins
Pamela Austin
Alisha Weir
Etta McDaniel
Hans Brausewetter
Linda Marlowe
Edith Barrett
Andrea Bowen
Jeff Winkless
Elisa Sednaoui
John Williams
Milos Kopecký
Billy Halop
Carl-Axel Heiknert
Jatin Sapru
Hiroshi Arikawa
Theresa Harris
Harry Rosenthal
Kiyoshi Kodama
Linda Clark
Matt Bettinelli-Olpin
Paulo Gustavo
Jesse Carere
Ester Dean
Josefin Crafoord
Myrna Hansen
Siobhan Fallon Hogan
Roy Castle
Isabelle Weingarten
Samy Seghir
Richard Farnsworth
Lynne Moody
Marina Schiptjenko
James Booth
Jane Hylton
Jessica De Gouw
Diane Morgan
Stephen Blackeh

Bobby Henrey
Mitchell Butel
Jim Hess
Ayda Aksel
Mino Bellei
Bill Travers
Michael Dorn
Ravi Dubey
Alessandra Mastronardi
José Marco Davó
Erika Toda
Baoguo Chen
Steven Mackintosh
Arsinée Khanjian
Ajiona Alexus
Cora Sue Collins
Sabina Franklyn
Jodie Turner-Smith
Brenda Song
Mark Patton
Gower Champion
Patsy Lynn
David Brain
Bob Hite
Max Harwood
Mia Lyhne
Carlos Bardem
Richard David-Caine
Elena Irureta
Vera Graziadei
Bethany Hamilton
Irandhir Santos
Avner W. Less
Alice Cooper
Jill Teed
Margaret McWade
Miguel Fernandes
Maksim Leonidov
John Franklin
Wood Harris
Sajid Yahiya
Theo Kogan
Gretchen Young
Zendaya
Lexi Atkins
Nicky Whelan
Sofía Reyes
Paul Meurisse
Byron Howard
Roman Madyanov
Amanda Wilson
Tony O'Dell
Paul Ritter
Richard Lumsden
Muse Watson
Donya Feuer
Greta Meyer
Noah Ringer
Joe W. Davis
Sinjin Smith
Roberts Blossom
Anthony Perkins
Oleg Skripka
Rockliffe Fellowes
Ray Abruzzo
Suzuka Ohgo
Jack Quaid
Andrew Leeds
Brett Marx
Patti LaBelle
Dick Simmons
Francesco Procopio
Yutaka Aoyama
Ke

Jerry Reed
Scott Ferrall
Kip King
Massimo Popolizio
Lindze Letherman
Jet Jurgensmeyer
Babul Supriyo
Kate Fitzgerald
Ebrahim Amini
Kim Hye-jun
Jalaja
Charles Berling
Michael Friend
John Lloyd Young
Justin Bell
Sandra Sade
Irwin Corey
Austin Pendleton
Na Li
Lily Cole
Jorge Martín
Harshitha
Brittney Powell
Frank Molinaro
Ingjerd Egeberg
Belinda Emmett
E.J. Bonilla
Silvana Gallardo
Jonathan Marc Sherman
China Lee
Saki Takaoka
Ana Amélia
Anita Björk
Kang Ho
Bradley Page
Barnard Hughes
Vladimir Tica
Yoon Kyung-ho
Garry Shandling
Petr Jákl
Yordan Bikov
Summer Naomi Smart
Steinunn Ólína Þorsteinsdóttir
Daisy Haggard
John DeSantis
Carlo Tamberlani
Pat Hurst
Malik Sealy
Frankie Thorn
Shelby Lynne
Hyuna
Leslie Sarony
Robert Toupin
Arne Bang-Hansen
Werner Brix
Ashika Ranganath
Lawrence Dane
Anatoliy Solonitsyn
Wolfgang Völz
Tao Liu
Kristen Wiig
Hiroaki Iwanaga
Nadège Beausson-Diagne
Ashley Blaine Featherson-Jenkins
Olga Dickie
Wade MacNeil
Sam Sanders
Rehan Sheikh
June Gable
Gary J. Tunnicliffe
Pe

Bruno Langley
Linda Hamilton
Alison Thornton
Cindy Vela
Parker Fennelly
Ingrid Olava
Aly Michalka
Lanna Saunders
Natacha Régnier
Helena Kallianiotes
Jo Sung-ha
Anu Emmanuel
Eddie Foy III
Richard Loring
Jeremy Sims
Alex D. Linz
John Christopher Jones
Jerry Miller
Travis Fimmel
Roebuck 'Pops' Staples
David Whiteley
Cara DeLizia
R.G. Armstrong
Dana Wheeler-Nicholson
Assen Blatechki
Marilyn Manson
Jaqueline Fleming
François Chau
Miguel Zúñiga
Ann Jillian
João Baldasserini
Kanika Tiwari
Max Shulman
Joyce Chu
William Daniels
Lyndsy Fonseca
Dorothy Phillips
Anne Archer
Orso Maria Guerrini
Rick Batalla
Olegar Fedoro
Vladek Sheybal
Florence Brudenell-Bruce
Joivan Wade
Barbara Tyson
Waylon Jennings
Pavel Volya
Radhika Apte
Richard Steinmetz
Umashree
Andries Brink
Ian Lindsay
Henric Holmberg
Claudio Botosso
Yui Natsukawa
Niamh Wilson
Jim Sturgess
Frank Bonner
Jason Finn
Brenda de Banzie
David Brin
Nidal Al-Askhar
Faraz Khan
Tilo Prückner
Jimmy Carter
Biplab Chatterjee
Terri Welles
David Salles
Ja

,Name,Birthdate
0,Alaska L. McFadden,NaN
1,Meredith MacNeill,1975-06-19
2,Ruth Trouncer,NaN
3,Christopher Aydon,NaN
4,Rick Marshall,NaN
...,...,...
181708,Connee Boswell,1907-12-03
181709,Stephanie March,1974-07-23
181710,Joe Pistone,1939-09-17
181711,Gayle Hunnicutt,1943-02-06


In [11]:
# deleting nans from our df
df_birthdays_notnan = df_birthdays[df_birthdays['Birthdate'].notnull()].copy().reset_index(drop=True)
df_birthdays_nan = df_birthdays[df_birthdays['Birthdate'].isna()].copy().reset_index(drop=True)

In [12]:
df_birthdays_notnan.to_csv("df_birthdays_movies.csv")

In [13]:
df_birthdays_notnan

,Name,Birthdate
0,Meredith MacNeill,1975-06-19
1,Melvin Van Peebles,1932-08-21
2,Michael Starke,1957-11-13
3,Ralf Wolter,1926-11-26
4,Beate Finckh,1960-04-22
...,...,...
41993,Samm Levine,1982-03-12
41994,Connee Boswell,1907-12-03
41995,Stephanie March,1974-07-23
41996,Joe Pistone,1939-09-17


----
----